HemoPheno4HF  
SCRIPT DESCRIPTION: Training process for developing MVDDs  
CODE DEVELOPED BY: Josephine Lamp  
ORGANIZATION: University of Virginia, Charlottesville, VA  
LAST UPDATED: 8/24/2020  

In [3]:
from MVDD.MVDD import MVDD
import MVDD.MVDD_Generator as mvGen
import networkx as nx
from networkx.drawing.nx_pydot import *
import Params as params
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import random
from itertools import permutations
import warnings
warnings.filterwarnings('ignore')

# Hemo

In [36]:
#Load data
hemoData = pd.read_csv('Data/Preprocessed Data/Cluster_Hemo.csv', index_col='DEIDNUM')
allScores = hemoData['Score']
death = hemoData['ScoreDeath']
rehosp = hemoData['ScoreRehosp']
readm = hemoData['ScoreReadmission']

# Preprocess and create training and testing sets
hemo = hemoData.drop('Score', axis=1)
hemo = hemo.drop('ScoreDeath', axis=1)
hemo = hemo.drop('ScoreRehosp', axis=1)
hemo = hemo.drop('ScoreReadmission', axis=1)
hemo = hemo.replace(np.inf, 0)
hemo = hemo.fillna(0)
xTrain, xTest, yTrain, yTest = train_test_split(hemo, allScores, test_size=.2)

print(xTrain.shape, xTest.shape)
xTrain

(334, 27) (84, 27)


,RAP,PAS,PAD,PAMN,PCWP,PCWPMod,PCWPA,PCWPMN,CO,CI,...,MPAP,CPI,PP,PPP,PAPP,SVR,RAT,PPRatio,Age,EjF
DEIDNUM,,,,,,,,,,,,,,,,,,,,,
79447,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,57.0,25.0
21937,5.0,27.0,15.0,20.0,0.0,15.0,0.0,0.0,0.00,0.00,...,37.000000,0.000000,36.0,0.409091,0.444444,0.000000,0.000000,0.400000,70.0,29.0
97047,5.0,40.0,20.0,25.0,18.0,18.0,18.0,18.0,4.51,2.22,...,53.333333,0.733437,39.0,0.371429,0.500000,2554.323725,0.277778,0.448276,54.0,40.0
16244,12.0,57.0,33.0,41.0,30.0,30.0,30.0,30.0,2.80,1.60,...,79.000000,0.745011,50.0,0.342466,0.421053,5657.142857,0.400000,0.555556,61.0,10.0
75134,22.0,64.0,24.0,37.0,22.0,22.0,22.0,0.0,2.86,1.45,...,80.000000,0.571212,61.0,0.465649,0.625000,4354.312354,1.000000,0.859155,78.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29783,15.0,75.0,35.0,48.0,35.0,35.0,35.0,0.0,7.56,3.50,...,98.333333,0.957132,35.0,0.397727,0.533333,1146.384480,0.428571,0.437500,68.0,30.0
89539,16.0,38.0,17.0,23.0,16.0,16.0,16.0,0.0,25.00,1.50,...,49.333333,0.450111,42.0,0.428571,0.552632,381.866667,1.000000,0.575342,76.0,45.0
81,10.0,40.0,20.0,27.0,18.0,18.0,18.0,20.0,4.52,2.02,...,53.333333,0.546430,17.0,0.212500,0.500000,1982.300885,0.555556,0.242857,69.0,20.0


In [10]:
def getParamAccuracy(treeFilename, rootNode, xData, yData, paramRanges, relops):
    dot = read_dot(treeFilename + '.dot')
    dot = nx.DiGraph(dot)
    mvdd = MVDD(params.hemo, dot, root=rootNode)
    mvdd.featureDict = params.hemoDict

    mvParam, usedParams, usedRelops = mvGen.addGraphParams(mvdd, paramRanges, relops, inorder=True)

    mvParam.saveToFile(treeFilename + "Params")
    mvParam.saveDotFile(treeFilename + "Params")

    predScores = []

    for index, row in xData.iterrows():
        score, path = mvParam.predictScore(row)
        predScores.append(int(score))

    acc = accuracy_score(yData, predScores)

    return acc

In [37]:
#Working here, find better parameter estimation method
featureRanges = params.hemoDict
treeAccDF = pd.read_csv("AccuracyDFTrees.csv")

origacc = float(treeAccDF[treeAccDF['Filename'] == filename + '.dot']['Accuracy'])
print("Original Accuracy was", origacc)

#get total number of branches for each node
branchDict = mvdd.getNumberBranchesPerNode(returnTerminals=False)


#based on number of branches per node, pick a split of values in the variable range
paramDict = {}
for k in branchDict.keys():
    numBranches = branchDict[k]
    low = featureRanges[k][0]
    high = featureRanges[k][1]

    lst = []
    for n in range(numBranches):
        lst.append(random.uniform(low, high))

    paramDict[k] = lst

# Run param optimization
filename = 'TreeFiles/TreeTraining/treeRandom37'

#     origacc = float(treeAccDF[treeAccDF['Filename'] == filename + '.dot']['Accuracy'])
#     print("Original Accuracy was", origacc)

acc = getParamAccuracy(treeFilename='TreeFiles/TreeTraining/treeRandom37', rootNode='PCWP', xData=xTrain,
                                             yData=yTrain, paramRanges=params.clusterHemoScoreDict, relops=params.hemoRelopsV1)
print("Accuracy is", acc)

Original Accuracy was 0.4910179640718562
Accuracy is 0.15868263473053892


# All Data

In [28]:
#Load data
allDataOrig = pd.read_csv('Data/Preprocessed Data/Cluster_AllData.csv', index_col='DEIDNUM')
allScores = allDataOrig['Score']
death = allDataOrig['ScoreDeath']
rehosp = allDataOrig['ScoreRehosp']
readm = allDataOrig['ScoreReadmission']

# Preprocess and create training and testing sets
allData = allDataOrig.drop('Score', axis=1)
allData = allData.drop('ScoreDeath', axis=1)
allData = allData.drop('ScoreRehosp', axis=1)
allData = allData.drop('ScoreReadmission', axis=1)
allData = allData.replace(np.inf, 0)
allData = allData.fillna(0)
xTrain, xTest, yTrain, yTest = train_test_split(allData, allScores, test_size=.2)

print(xTrain.shape, xTest.shape)
xTrain

(692, 118) (174, 118)


,Age,Gender,Race,Wt,BMI,InitialHospDays,TotalHospDays,NYHA,MLHFS,AF,...,VALSA,EjF,BPDIAS,BPSYS,HR,PV,MAP,PP,PPP,PPRatio
DEIDNUM,,,,,,,,,,,,,,,,,,,,,
53357,81.0,1.0,1.0,78.727273,27.893733,11.0,11.0,3.0,58.0,1.0,...,0.0,29.0,56.0,82.0,80.0,10.931617,119.333333,26.0,0.317073,0.325000
12868,72.0,1.0,1.0,69.090909,25.377744,12.0,17.0,3.0,66.0,1.0,...,0.0,14.0,63.0,96.0,78.0,0.000000,138.000000,33.0,0.343750,0.423077
5401,61.0,1.0,1.0,103.500000,34.581844,12.0,11.0,3.0,75.0,1.0,...,0.0,20.0,76.0,92.0,96.0,3.022408,142.666667,16.0,0.173913,0.166667
85589,50.0,1.0,1.0,114.000000,35.980306,8.0,66.0,4.0,65.0,1.0,...,0.0,15.0,34.0,80.0,57.0,15.887063,102.666667,46.0,0.575000,0.807018
1059,79.0,1.0,1.0,77.100000,26.678201,13.0,11.0,4.0,54.0,1.0,...,0.0,15.0,61.0,90.0,98.0,0.000000,130.666667,29.0,0.322222,0.295918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84987,56.0,1.0,4.0,54.100000,21.132812,6.0,9.0,4.0,80.0,0.0,...,0.0,20.0,72.0,117.0,110.0,-12.705110,165.000000,45.0,0.384615,0.409091
17343,49.0,2.0,2.0,98.727273,34.161686,4.0,10.0,4.0,67.0,0.0,...,0.0,15.0,73.0,114.0,58.0,-4.419776,162.666667,41.0,0.359649,0.706897
17905,69.0,1.0,1.0,82.000000,25.880571,3.0,3.0,2.0,61.0,1.0,...,0.0,5.0,50.0,100.0,70.0,-13.067916,133.333333,50.0,0.500000,0.714286
